# Development of create Excel Management accounts 
This is the first phase to create a template.

In [1]:
import datetime as dt
import os
import pandas as pd
import shutil
from xlsxwriter.utility import xl_rowcol_to_cell

In [19]:
class PeriodReport():
    """This holds the data for a single report.  This includees the historical and mtd figures as well as the
    P&L and Balance sheet data"""
    
    def __init__(self, period):
        """Period is a datetime for which the year and month are used"""
        self.period = period
        self.prev_period = period.replace(year = period.year - 1)
        self.year_start = period.replace(month = period.month - 1).replace(year = period.year - 1, month = 1)
        # Create a Pandas dataframe from the data.
        acct_index = [4000, 5000, 5001, 4009, 7000, 7204, 8405]
        self.nc_names = pd.DataFrame({'NC_Name': ['Sales', 'Materials Cost', 'Material cost Adjustment', 'Discounts Allowed',
                                              'Factory Wages', 'Factory Rent & Rates', 'Motor Running']}, 
                                     index = acct_index)
        self.period_mtd = pd.DataFrame({'TB': [10, 20, 30, 20, 15, 30, 45]}, index = acct_index)
        self.period_ytd = self.period_mtd
        self.prior_mtd = self.period_mtd 
        self.prior_ytd = self.period_mtd
        self.df_list = [self.period_mtd, self.prior_mtd, self.period_ytd, self.prior_ytd]

    @property
    def datestring(self):
        return self.period.strftime('%b %y')

    @property
    def long_datestring(self):
        return self.period.strftime('%B %y')

    @property
    def prev_datestring(self):
        return self.prev_period.strftime('%b %y')
    
    @property
    def year_start_string(self):
        return self.year_start.strftime('%b %y')

In [24]:
class ExcelManagementReport():
    
    def report_filename(self):
        return 'Management Report {}.xlsx'.format(dt.datetime.today().strftime('%Y-%m-%dT%H_%M_%S'))
    
    def __init__(self, rep):
        # Setup
        self.rep = rep
        self.col_list=(2, 3, 5, 6)
        # Action
        self.create_report()

        
    def write_row(self, ws, entries):
        # Add titles
        for i,column in enumerate((2, 3, 5, 6)):
            # Determine where we will place the formula
            cell_location = xl_rowcol_to_cell(self.line_number, column)
            ws.write(cell_location, entries[i], self.bold_fmt)
        self.line_number += 1

    def write_block(self, ws, sum_list, acct_list, title):
        block_sum = [0, 0, 0, 0]
        # TODO The account list may have entries for which there is no data.  Under these circumstances
        # the aim is to leave out those columns in the reporting.
        # This will depend on how the trial balance is shown with no data.
        if len(acct_list) == 1: # Single row so no summary row
            fmt = self.bold_fmt
            fmt_left = self.bold_fmt
        else:  
            fmt = self.fmt
            fmt_left = self.left_fmt
        
        for i, a in enumerate(acct_list):
            # If there no row then ignore error
            try:
                name = self.rep.nc_names.ix[acct_list[i]].NC_Name #Error will show up
                cell_location = xl_rowcol_to_cell(self.line_number, 0)
                ws.write(cell_location, acct_list[i], self.fmt)
                cell_location = xl_rowcol_to_cell(self.line_number, 1)
                ws.write(cell_location, name, fmt_left)
                for j, df in enumerate(self.rep.df_list):
                    cell_location = xl_rowcol_to_cell(self.line_number, self.col_list[j])
                    try:
                        value=df.ix[acct_list[i]].TB
                    except KeyError:
                        # There is a name value so presumably some data but just none in this series
                        value=0                        
                    ws.write(cell_location, value, fmt)
                    sum_list[j] += value
                self.line_number += 1
            except KeyError:
                # This is where there is no data in the name 
                print("Missing data for account {}".format(acct_list[i]))
        # Add a sub total line if required
        if len(acct_list) != 1:  
            for i,c in enumerate(self.col_list):
                cell_location = xl_rowcol_to_cell(self.line_number, c)
                ws.write(cell_location, block_sum[i], self.fmt)
            cell_location = xl_rowcol_to_cell(self.line_number, 1)
            ws.write(cell_location, title, self.bold_fmt)
            self.line_number += 1
        # Aggregate the local sum into the bigger sum
        for i,e in enumerate(block_sum):
            sum_list[i]+=e
        self.line_number += 1  # Blank line seperator
    
    def write_sum(self, ws, sum_list, title):
        cell_location = xl_rowcol_to_cell(self.line_number, 1)
        ws.write(cell_location, title, self.bold_left_fmt)
        for i, df in enumerate(self.rep.df_list):
            cell_location = xl_rowcol_to_cell(self.line_number, self.col_list[i])
            ws.write(cell_location, sum_list[i], self.bold_fmt)
        self.line_number += 2
    
    def format_pnl(self, wb, ws):
        # Nominal code info columns
        ws.set_column('A:A', 8.5)
        # Description column
        ws.set_column('B:B', 30)
        # This years figures
        ws.set_column('C:D', 11.5)
        # Margin
        ws.set_column('E:E', 7)
        # Historic figures
        ws.set_column('F:G', 11.5)
        # Total formatting
        fmt = {'align': 'center', 'font_name': 'Arial', 'font_size': 10}
        self.fmt = wb.add_format(fmt)
        self.left_fmt = wb.add_format({**fmt, **{'align': 'left'}})
        self.bold_fmt = wb.add_format({**fmt, **{'bold': True}})
        self.bold_left_fmt = wb.add_format({**fmt, **{'align': 'left', 'bold': True,}})
        self.line_number=0
        self.write_row(ws, [self.rep.datestring, self.rep.prev_datestring, self.rep.datestring, self.rep.prev_datestring])
        ws.write('A2', 'From End of Year ({})'.format(self.rep.year_start_string), self.bold_left_fmt)
        self.write_row(ws, ['PERIOD', 'PERIOD', 'YTD', 'YTD'])
        self.write_row(ws, ['£', '£', '£', '£'])
        self.profit_list = [0, 0, 0, 0]
        self.expense_list = [0, 0, 0, 0]
        self.line_number=4
        self.write_block(ws, self.profit_list, [4000], 'Sales')
        self.write_block(ws, self.expense_list, [5000, 5001], 'Total Material Cost')
        self.write_block(ws, self.expense_list, [7000, 7100, 7103, 7102, 7105, 7006], 'Variable Works Expense')
        self.write_block(ws, self.expense_list, [7200, 7202, 7204, 7206], 'Fixed Works Expenses')
        self.write_block(ws, self.expense_list, [7020, 8100, 8200, 8204, 8300, 7906, 8310, 8400, 8402, 8405, 8201,
                                                8433, 8408, 8410, 8414, 8420, 8424, 8426, 8430, 8435, 8440], 'Admin Expenses')
        self.write_block(ws, self.expense_list, [4905, 6100, 6200, 6201, 4009], 'Selling Expenses')
        self.write_sum(ws, self.expense_list, 'TOTAL EXPENSES')
        self.write_sum(ws, self.expense_list, 'PROFIT/(LOSS)')
        # Format for printing
        ws.print_area(0, 0, self.line_number, 6)
        header = '&LSLUMBERFLEECE' + '&CPROFIT & LOSS ACCOUNT' + '&R{}'.format(self.rep.long_datestring)
        footer = '&L&F' + '&R&D: &T'
        ws.set_header(header)
        ws.set_footer(footer)
        # Set A4 paper
        ws.set_paper(9)
        ws.hide_gridlines(0)
        ws.fit_to_pages(1,1)  # Fit to one page

    
    def create_report(self):
        fn = os.getcwd() + '\\' + self.report_filename()
        if os.path.isfile(fn):
            shutil.removefile(fn)
        # Create a Pandas Excel writer using XlsxWriter as the engine.
        writer = pd.ExcelWriter(fn, engine='xlsxwriter')
        # Get the xlsxwriter objects from the dataframe writer object.
        workbook  = writer.book
        self.sheetname = 'P&L'
        worksheet = workbook.add_worksheet(self.sheetname)
        #self.rep.df.to_excel(writer,)
        self.format_pnl(workbook, worksheet)
        writer.save()

In [25]:
rep = PeriodReport(dt.datetime(2016,2,1))
ExcelManagementReport(rep)
rep.prior_mtd.head()

Missing data for account 7100
Missing data for account 7103
Missing data for account 7102
Missing data for account 7105
Missing data for account 7006
Missing data for account 7200
Missing data for account 7202
Missing data for account 7206
Missing data for account 7020
Missing data for account 8100
Missing data for account 8200
Missing data for account 8204
Missing data for account 8300
Missing data for account 7906
Missing data for account 8310
Missing data for account 8400
Missing data for account 8402
Missing data for account 8201
Missing data for account 8433
Missing data for account 8408
Missing data for account 8410
Missing data for account 8414
Missing data for account 8420
Missing data for account 8424
Missing data for account 8426
Missing data for account 8430
Missing data for account 8435
Missing data for account 8440
Missing data for account 4905
Missing data for account 6100
Missing data for account 6200
Missing data for account 6201


,TB
4000,10
5000,20
5001,30
4009,20
7000,15
